Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,roc_auc_score,mean_squared_error,f1_score,recall_score,precision_score
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from numpy import argmax
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool, cv
from lightgbm import LGBMClassifier


In [12]:
SEED = 0

In [13]:
# load datasets
train = pd.read_csv("hr_train.csv")
test = pd.read_csv("hr_test.csv")
train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [7]:
test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [9]:
train.describe()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
count,54808.000000,54808.000000,54808.000000,50684.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000
mean,39195.830627,1.253011,34.803915,3.329256,5.865512,0.351974,0.023172,63.386750,0.085170
std,22586.581449,0.609264,7.660169,1.259993,4.265094,0.477590,0.150450,13.371559,0.279137
min,1.000000,1.000000,20.000000,1.000000,1.000000,0.000000,0.000000,39.000000,0.000000
25%,19669.750000,1.000000,29.000000,3.000000,3.000000,0.000000,0.000000,51.000000,0.000000
50%,39225.500000,1.000000,33.000000,3.000000,5.000000,0.000000,0.000000,60.000000,0.000000
75%,58730.500000,1.000000,39.000000,4.000000,7.000000,1.000000,0.000000,76.000000,0.000000
max,78298.000000,10.000000,60.000000,5.000000,37.000000,1.000000,1.000000,99.000000,1.000000


In [13]:
train.isna().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [15]:
values={"previous_year_rating":3,"education":"O"}
train.fillna(value=values,inplace=True)

In [17]:
categorical_features_indices = np.array([0,1,2,3,4,6,8,10,11])
data_xs = train.drop(['employee_id',"is_promoted"],axis=1)

In [18]:
x_train,x_test,y_train,y_test = train_test_split(data_xs,train["is_promoted"], test_size = 0.3,random_state=20)

In [19]:
def model_evalution(model):
  print("####################### model Evalution started #######################")
  train_pre = model.predict(x_train)
  test_pre = model.predict(x_test)
  train_pro = model.predict_proba(x_train)
  test_pro = model.predict_proba(x_test)
  
  print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
  print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
  print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
  print("Train F1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
  print("Train recall: {0} \t Test recall: {1}".format(recall_score(y_train, train_pre),recall_score(y_test,test_pre)))
  print("Train precision: {0} \t Test Precision: {1}".format(precision_score(y_train, train_pre),precision_score(y_test,test_pre)))
  print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  #f1_score

In [20]:
catboost_model = CatBoostClassifier(iterations=400,verbose=True,eval_metric="F1",learning_rate=0.2,class_weights=[1,3],depth=3,l2_leaf_reg=100,bagging_temperature=0.5)
catboost_model.fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test),use_best_model=False)
model_evalution(catboost_model)

0:	learn: 0.2017291	test: 0.2112933	best: 0.2112933 (0)	total: 86.1ms	remaining: 34.4s
1:	learn: 0.2017291	test: 0.2112933	best: 0.2112933 (0)	total: 101ms	remaining: 20.2s
2:	learn: 0.2884808	test: 0.2778105	best: 0.2778105 (2)	total: 135ms	remaining: 17.9s
3:	learn: 0.4029994	test: 0.4097015	best: 0.4097015 (3)	total: 162ms	remaining: 16.1s
4:	learn: 0.4983911	test: 0.5069238	best: 0.5069238 (4)	total: 195ms	remaining: 15.4s
5:	learn: 0.5049527	test: 0.5081157	best: 0.5081157 (5)	total: 212ms	remaining: 13.9s
6:	learn: 0.5095344	test: 0.5093343	best: 0.5093343 (6)	total: 232ms	remaining: 13s
7:	learn: 0.5085426	test: 0.5093343	best: 0.5093343 (6)	total: 253ms	remaining: 12.4s
8:	learn: 0.5082101	test: 0.5094240	best: 0.5094240 (8)	total: 267ms	remaining: 11.6s
9:	learn: 0.5045180	test: 0.5145563	best: 0.5145563 (9)	total: 300ms	remaining: 11.7s
10:	learn: 0.5097400	test: 0.5201044	best: 0.5201044 (10)	total: 328ms	remaining: 11.6s
11:	learn: 0.5129683	test: 0.5201044	best: 0.5201044 

97:	learn: 0.5632925	test: 0.5593054	best: 0.5597381 (96)	total: 2.77s	remaining: 8.53s
98:	learn: 0.5635130	test: 0.5593054	best: 0.5597381 (96)	total: 2.81s	remaining: 8.55s
99:	learn: 0.5644860	test: 0.5585970	best: 0.5597381 (96)	total: 2.85s	remaining: 8.55s
100:	learn: 0.5668050	test: 0.5585970	best: 0.5597381 (96)	total: 2.88s	remaining: 8.54s
101:	learn: 0.5659124	test: 0.5601048	best: 0.5601048 (101)	total: 2.92s	remaining: 8.52s
102:	learn: 0.5661709	test: 0.5593970	best: 0.5601048 (101)	total: 2.93s	remaining: 8.45s
103:	learn: 0.5650038	test: 0.5611511	best: 0.5611511 (103)	total: 2.96s	remaining: 8.42s
104:	learn: 0.5652234	test: 0.5611511	best: 0.5611511 (103)	total: 2.99s	remaining: 8.4s
105:	learn: 0.5651452	test: 0.5611511	best: 0.5611511 (103)	total: 3s	remaining: 8.33s
106:	learn: 0.5651061	test: 0.5612428	best: 0.5612428 (106)	total: 3.02s	remaining: 8.27s
107:	learn: 0.5655058	test: 0.5612428	best: 0.5612428 (106)	total: 3.04s	remaining: 8.22s
108:	learn: 0.5659986

193:	learn: 0.5730522	test: 0.5688488	best: 0.5698595 (176)	total: 4.61s	remaining: 4.89s
194:	learn: 0.5730522	test: 0.5688488	best: 0.5698595 (176)	total: 4.63s	remaining: 4.87s
195:	learn: 0.5731699	test: 0.5689405	best: 0.5698595 (176)	total: 4.66s	remaining: 4.85s
196:	learn: 0.5735989	test: 0.5690821	best: 0.5698595 (176)	total: 4.67s	remaining: 4.81s
197:	learn: 0.5738916	test: 0.5690821	best: 0.5698595 (176)	total: 4.69s	remaining: 4.79s
198:	learn: 0.5738916	test: 0.5690821	best: 0.5698595 (176)	total: 4.7s	remaining: 4.75s
199:	learn: 0.5739309	test: 0.5690821	best: 0.5698595 (176)	total: 4.72s	remaining: 4.72s
200:	learn: 0.5741843	test: 0.5686654	best: 0.5698595 (176)	total: 4.73s	remaining: 4.68s
201:	learn: 0.5741843	test: 0.5686654	best: 0.5698595 (176)	total: 4.75s	remaining: 4.65s
202:	learn: 0.5741843	test: 0.5686654	best: 0.5698595 (176)	total: 4.76s	remaining: 4.62s
203:	learn: 0.5741843	test: 0.5686654	best: 0.5698595 (176)	total: 4.78s	remaining: 4.59s
204:	learn:

292:	learn: 0.5767897	test: 0.5711538	best: 0.5746722 (269)	total: 6.52s	remaining: 2.38s
293:	learn: 0.5767897	test: 0.5711538	best: 0.5746722 (269)	total: 6.54s	remaining: 2.36s
294:	learn: 0.5767897	test: 0.5711538	best: 0.5746722 (269)	total: 6.55s	remaining: 2.33s
295:	learn: 0.5768289	test: 0.5710623	best: 0.5746722 (269)	total: 6.57s	remaining: 2.31s
296:	learn: 0.5774093	test: 0.5723431	best: 0.5746722 (269)	total: 6.58s	remaining: 2.28s
297:	learn: 0.5776208	test: 0.5721600	best: 0.5746722 (269)	total: 6.6s	remaining: 2.26s
298:	learn: 0.5771347	test: 0.5717941	best: 0.5746722 (269)	total: 6.62s	remaining: 2.24s
299:	learn: 0.5770563	test: 0.5717941	best: 0.5746722 (269)	total: 6.64s	remaining: 2.21s
300:	learn: 0.5790830	test: 0.5729799	best: 0.5746722 (269)	total: 6.67s	remaining: 2.19s
301:	learn: 0.5791223	test: 0.5729799	best: 0.5746722 (269)	total: 6.68s	remaining: 2.17s
302:	learn: 0.5791223	test: 0.5729799	best: 0.5746722 (269)	total: 6.7s	remaining: 2.15s
303:	learn: 

389:	learn: 0.5817051	test: 0.5742038	best: 0.5757865 (371)	total: 8.6s	remaining: 220ms
390:	learn: 0.5817444	test: 0.5742038	best: 0.5757865 (371)	total: 8.62s	remaining: 198ms
391:	learn: 0.5817838	test: 0.5729316	best: 0.5757865 (371)	total: 8.65s	remaining: 177ms
392:	learn: 0.5815871	test: 0.5728403	best: 0.5757865 (371)	total: 8.67s	remaining: 154ms
393:	learn: 0.5833614	test: 0.5754732	best: 0.5757865 (371)	total: 8.7s	remaining: 132ms
394:	learn: 0.5828386	test: 0.5746102	best: 0.5757865 (371)	total: 8.73s	remaining: 111ms
395:	learn: 0.5828780	test: 0.5766481	best: 0.5766481 (395)	total: 8.77s	remaining: 88.6ms
396:	learn: 0.5825911	test: 0.5768314	best: 0.5768314 (396)	total: 8.79s	remaining: 66.5ms
397:	learn: 0.5831647	test: 0.5767398	best: 0.5768314 (396)	total: 8.82s	remaining: 44.3ms
398:	learn: 0.5836875	test: 0.5747419	best: 0.5768314 (396)	total: 8.85s	remaining: 22.2ms
399:	learn: 0.5839741	test: 0.5747419	best: 0.5768314 (396)	total: 8.88s	remaining: 0us

bestTest 

In [24]:
values={"previous_year_rating":3,"education":"O"}
test.fillna(value=values,inplace=True)

In [26]:
data_test_xs = test.drop(['employee_id'],axis=1)
y_test_pre = catboost_model.predict(data_test_xs)

In [27]:
test["is_promoted"] = y_test_pre

In [34]:
test.to_csv("analytics_submission.csv",columns=["employee_id","is_promoted"],index=False)

In [35]:
submission = pd.read_csv("analytics_submission.csv")
submission['is_promoted'].value_counts()

0.0    22328
1.0     1162
Name: is_promoted, dtype: int64